## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [17]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib qt

def store_image(img, img_dir, fname):
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
    img_path = img_dir + fname.split('/')[-1]
    #print(img_path)
    cv2.imwrite(img_path, img)
        
def calibrate_camera():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #store every images with detected corners for post analyse and writeup
            img_dir = 'output_images/chessboard_corners/'
            img_path = img_dir + fname.split('/')[-1]
            store_image(img, img_dir, img_path)
    #calibrate camera with identified objpoints and imgpoints
    img_size = (img.shape[1], img.shape[0])
    return cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img_copy = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img_copy, cv2.COLOR_RGB2HLS).astype(np.float)
    #hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float)
    #l_channel = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #hsv[:,:,1]
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

def transform(img, src, dst):
    # Compute and apply perpective transform
    #vertices = np.array([[(160,700), (570, 460),(740, 460), (1150,700)]], dtype=np.int32)
    #cv2.polylines(img, vertices, 1, color=(255, 0, 0), thickness=2)
 
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size)  # keep same size as input image
    
    return warped

def undistort(img):
    return cv2.undistort(img, mtx, dist, None, mtx)
    

# Calibrate camera

In [18]:
ret, mtx, dist, rvecs, tvecs = calibrate_camera()

## Test pipeline on test images and store output

In [19]:
# Make a list of calibration images
images = glob.glob('test_images/*.jpg')
#define area of interest
#src = np.float32([[570,460],[740,460],[160,700],[1120,700]])
#dst
#dst = np.float32([[560,460],[750,460],[150,700],[1130,700]])
img = cv2.imread('test_images/test1.jpg')
W = img.shape[1]
H = img.shape[0]

print('W:', W, 'H:', H)

# Source coordinates
src = np.float32([[W * 0.4475, H * 0.65],
                [W * 0.5525, H * 0.65],
                [W * 0.175, H * 0.95],
                [W * 0.825, H * 0.95]])
print('src:', src)    
# Destination coordinates
dst = np.float32([[W * 0.2, H * 0.025],
                [W * 0.8, H * 0.025],
                [W * 0.2, H * 0.975],
                [W * 0.8, H * 0.975]])
print('dst:', dst)
# process test images
for fname in images:
    img = cv2.imread(fname)
    undist_img = undistort(img)
    store_image(undist_img, 'output_images/undistorted/', fname)
    binary_image = pipeline(undist_img)
    store_image(binary_image*255, 'output_images/binary_images/', fname)
    transformed = transform(binary_image, src, dst)
    store_image(transformed*255, 'output_images/transformed_images/', fname)

plt.figure(figsize=(12, 7))
plt.subplot(2, 2, 1)
plt.imshow(img)
plt.title("Original Image")
plt.subplot(2, 2, 2)
plt.imshow(undist_img)
plt.title("Undistorted Image")
#img = cv2.imread('test_images/test1.jpg')
plt.subplot(2, 2, 3)
plt.imshow(binary_image*255)
plt.title("Binary Image")
plt.subplot(2, 2, 4)
plt.imshow(transformed*255)
plt.title("Transformed Image")
#plt.savefig("readme_img/undist_img.jpg")

# Plot the result
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()

#ax1.imshow(img, cmap='gray')
#ax1.set_title('Original Image', fontsize=40)

#ax2.imshow(binary_image, cmap='gray')
#ax2.set_title('Pipeline Result', fontsize=40)

#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
#show last image for 3 sec for immediate control
#cv2.imshow('warped',warped)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
    


W: 1280 H: 720
src: [[  572.79998779   468.        ]
 [  707.20001221   468.        ]
 [  224.           684.        ]
 [ 1056.           684.        ]]
dst: [[  256.    18.]
 [ 1024.    18.]
 [  256.   702.]
 [ 1024.   702.]]
